In [115]:
#importing libraries
import numpy as np
from collections import Counter
import pandas as pd
from sklearn.metrics import mean_squared_error,roc_auc_score,precision_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, confusion_matrix, log_loss,roc_auc_score
from sklearn.multiclass import OneVsRestClassifier

# for lightGBM
from lightgbm import LGBMClassifier

# for naive bayes
from sklearn.naive_bayes import GaussianNB

# for evaluation metrics
%run -i helper_functions.py

# for multi-label classification
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset
from sklearn.multiclass import OneVsRestClassifier

In [54]:
#loading the train dataset
selected_train = pd.read_csv('Data/selected_train.csv')
selected_train.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate,none,min_length_scaled,num_words_vs_length,exclamation_marks_vs_length,num_unique_words_scaled,verbs_vs_length,num_uppercase_scaled,uppercase_vs_length,sentiment,bad_toxic_vs_length,bad_severe_toxic_vs_length,bad_obscene_vs_length,bad_threat_vs_length,bad_insult_vs_length,bad_identity_hate_vs_length,29,34,46,47,53,54,65,72,82,86,87,93,95,96,98,100,103,105,114,132,135,139,143,156,157,170,198
0,0,0,0,0,0,0,1,0.009393,0.181132,0.000000,0.106329,0.041509,0.003426,0.064151,0.5574,0.0,0.0,0.0,0.0,0.0,0.0,-0.054548,-0.105844,-0.023651,-0.120931,0.029694,0.000656,0.138915,-0.011885,0.066578,-0.030327,-0.030858,0.003075,-0.001155,-0.074290,0.075012,-0.009385,0.039875,-0.091320,-0.017755,0.003997,-0.199211,-0.109984,0.089084,0.012651,-0.019501,-0.091946,-0.113697
1,0,0,0,0,0,0,1,0.000723,0.160714,0.008929,0.043038,0.026786,0.001612,0.071429,0.2942,0.0,0.0,0.0,0.0,0.0,0.0,0.104774,-0.020276,0.102398,-0.146403,0.014844,0.019279,0.066769,0.092921,-0.116742,0.059807,-0.077983,-0.097744,-0.097991,0.028930,0.115281,0.009460,0.122598,-0.118445,0.039541,0.017522,0.001397,0.000094,0.021135,0.039308,-0.067295,-0.079366,-0.044959
2,0,0,0,0,0,0,1,0.007225,0.188841,0.000000,0.091139,0.038627,0.000806,0.017167,-0.1779,0.0,0.0,0.0,0.0,0.0,0.0,0.022507,-0.155533,0.175771,0.021757,-0.035649,-0.149408,0.170970,0.032876,0.040762,-0.089416,-0.162394,0.067722,-0.108610,-0.030535,0.072878,-0.015372,0.148674,-0.131555,-0.060714,0.026459,-0.029582,-0.146134,0.109907,0.036539,-0.053628,0.027530,-0.211230
3,0,0,0,0,0,0,1,0.007948,0.175719,0.000000,0.141772,0.036741,0.002217,0.017572,0.5106,0.0,0.0,0.0,0.0,0.0,0.0,0.028492,-0.103913,0.071215,-0.131409,0.066988,-0.070037,0.184324,0.047017,0.000166,-0.044984,-0.149924,0.042974,-0.038304,0.050073,0.022973,0.005240,0.091080,-0.145382,-0.004285,-0.000706,-0.134984,-0.214832,0.177560,0.007675,-0.087473,0.106848,-0.015987
4,0,0,0,0,0,0,1,0.003613,0.208955,0.000000,0.027848,0.059701,0.000403,0.029851,0.6808,0.0,0.0,0.0,0.0,0.0,0.0,0.078496,-0.047719,0.130365,-0.037535,-0.104838,0.131475,0.138012,0.020882,0.027521,-0.151158,-0.135344,0.050743,-0.000602,0.006453,-0.008789,-0.115800,0.190293,0.007073,-0.204627,0.162032,0.013798,-0.221076,0.163578,-0.068310,-0.031184,-0.017280,0.038566


In [46]:
#loading the test dataset
selected_test = pd.read_csv('Data/selected_test.csv')
selected_test.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate,min_length_scaled,num_words_vs_length,exclamation_marks_vs_length,num_unique_words_scaled,verbs_vs_length,num_uppercase_scaled,uppercase_vs_length,sentiment,bad_toxic_vs_length,bad_severe_toxic_vs_length,bad_obscene_vs_length,bad_threat_vs_length,bad_insult_vs_length,bad_identity_hate_vs_length,29,34,46,47,53,54,65,72,82,86,87,93,95,96,98,100,103,105,114,132,135,139,143,156,157,170,198,none
0,0,0,0,0,0,0,0.002890,0.166667,0.000000,0.032911,0.031250,0.000403,0.020833,0.3612,0.0,0.0,0.0,0.0,0.0,0.0,-0.052982,-0.169101,0.104210,-0.231872,-0.147303,0.085074,0.279414,0.045263,0.216464,-0.070892,-0.137326,0.045426,-0.138258,0.062082,0.168577,-0.144716,0.115447,-0.123190,0.040738,-0.010083,-0.158238,-0.204194,0.116565,0.098296,-0.206160,0.046779,-0.036748,1
1,0,0,0,0,0,0,0.005058,0.187500,0.000000,0.015190,0.031250,0.000202,0.031250,0.0516,0.0,0.0,0.0,0.0,0.0,0.0,0.032172,0.046765,0.194164,0.005340,-0.175857,0.152049,0.067513,0.093680,0.433347,-0.139590,-0.156423,0.379176,-0.015764,-0.018882,0.210565,-0.238187,0.241806,-0.004935,-0.013467,0.003550,-0.117174,-0.075475,0.043945,-0.148617,-0.350740,0.113815,-0.070907,1
2,0,0,0,0,0,0,0.000723,0.149554,0.002232,0.118987,0.031250,0.002821,0.031250,-0.7393,0.0,0.0,0.0,0.0,0.0,0.0,0.000621,-0.132685,0.156570,-0.122877,-0.041197,-0.045822,0.151609,-0.012794,0.058626,0.027053,-0.184707,0.032653,-0.047319,0.004221,0.065742,-0.074447,0.009821,-0.114241,-0.125080,0.045087,-0.119656,-0.274707,0.102546,0.100782,-0.107205,-0.050780,-0.098343,1
3,0,0,0,0,0,0,0.007225,0.169661,0.000000,0.070886,0.029940,0.008061,0.079840,0.0258,0.0,0.0,0.0,0.0,0.0,0.0,0.079728,0.022776,0.088192,-0.004380,0.004026,-0.220622,0.163259,0.237079,0.203803,-0.036898,-0.085977,0.060488,-0.030683,0.002059,0.092577,0.084728,-0.005106,-0.241482,-0.023304,-0.005002,-0.281848,-0.157011,0.184323,0.067625,-0.066242,0.023737,-0.111269,1
4,0,0,0,0,0,0,0.003613,0.167665,0.000000,0.060759,0.035928,0.000806,0.011976,0.7184,0.0,0.0,0.0,0.0,0.0,0.0,0.104646,-0.068364,-0.012882,-0.102120,0.041755,-0.031094,0.149613,0.158524,0.137380,0.068439,-0.086570,0.027189,-0.114363,0.001011,-0.029552,0.054780,0.124478,-0.192200,0.055657,0.024936,-0.181419,-0.165707,0.194511,0.143557,-0.059017,0.136676,-0.004530,1


In [59]:
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
features = selected_train.columns[7:].tolist()
len(features)

41

## Gradient Boosting (LGBMClassifier)

### Baseline Gradient Boosting using BinaryRelevance

In [61]:
classifier_gb = BinaryRelevance(LGBMClassifier(random_state=0))
# train
classifier_gb.fit(selected_train[features], selected_train[labels])
# predict
predictions_gb = classifier_gb.predict(selected_train[features])

In [62]:
# predict probability
predictions_proba_gb = classifier_gb.predict_proba(selected_train[features])

In [63]:
# model evaluation
get_evaluation_score(selected_train[labels], predictions_gb, predictions_proba_gb)

Accuracy score:  0.9324626655219307
Precision score:  0.8585256861860837
Recall score:  0.7201834862385321
F1 score:  0.7820014889765815
Confusion matrix for label toxic:
[[142777   1500]
 [  4464  10830]]
Confusion matrix for label severe_toxic:
[[157808    168]
 [   477   1118]]
Confusion matrix for label obscene:
[[150046   1076]
 [  1836   6613]]
Confusion matrix for label threat:
[[159055     38]
 [    50    428]]
Confusion matrix for label insult:
[[150390   1304]
 [  2335   5542]]
Confusion matrix for label identity_hate:
[[158060    106]
 [   659    746]]
Logarithmic Loss:  0.2437155281367502
ROC AUC score:  0.9860905444013781


In [64]:
# evaluation on test data
predictions_gb_test = classifier_gb.predict(selected_test[features])
predictions_proba_gb_test = classifier_gb.predict_proba(selected_test[features])
get_evaluation_score(selected_test[labels], predictions_gb_test, predictions_proba_gb_test)

Accuracy score:  0.8789896526931132
Precision score:  0.5829492803206758
Recall score:  0.6643674989653745
F1 score:  0.6174134411558834
Confusion matrix for label toxic:
[[54806  3082]
 [ 1502  4588]]
Confusion matrix for label severe_toxic:
[[63254   357]
 [  220   147]]
Confusion matrix for label obscene:
[[58600  1687]
 [ 1071  2620]]
Confusion matrix for label threat:
[[63381   386]
 [  156    55]]
Confusion matrix for label insult:
[[59351  1200]
 [ 1367  2060]]
Confusion matrix for label identity_hate:
[[62992   274]
 [  550   162]]
Logarithmic Loss:  0.3109336144201333
ROC AUC score:  0.9583558250719668


### Baseline Gradient Boosting using ClassifierChain

In [65]:
classifier_chain_gb = ClassifierChain(LGBMClassifier(random_state=0))
# train
classifier_chain_gb.fit(selected_train[features], selected_train[labels])
# predict
predictions_chain_gb = classifier_chain_gb.predict(selected_train[features])

In [66]:
# predict probability
predictions_proba_chain_gb = classifier_chain_gb.predict_proba(selected_train[features])

In [67]:
# model evaluation
get_evaluation_score(selected_train[labels], predictions_chain_gb, predictions_proba_chain_gb)

Accuracy score:  0.9344868428473846
Precision score:  0.8403138050122246
Recall score:  0.7235170095162118
F1 score:  0.775799218354555
Confusion matrix for label toxic:
[[142777   1500]
 [  4464  10830]]
Confusion matrix for label severe_toxic:
[[157818    158]
 [   458   1137]]
Confusion matrix for label obscene:
[[149876   1246]
 [  1717   6732]]
Confusion matrix for label threat:
[[158951    142]
 [   198    280]]
Confusion matrix for label insult:
[[150017   1677]
 [  2165   5712]]
Confusion matrix for label identity_hate:
[[157990    176]
 [   702    703]]
Logarithmic Loss:  0.27562349981689854
ROC AUC score:  0.98159005455481


In [69]:
# evaluation on test data
predictions_chain_gb_test = classifier_chain_gb.predict(selected_test[features])
predictions_proba_chain_gb_test = classifier_chain_gb.predict_proba(selected_test[features])
get_evaluation_score(selected_test[labels], predictions_chain_gb_test, predictions_proba_chain_gb_test)

Accuracy score:  0.8797868017130889
Precision score:  0.5570072116149407
Recall score:  0.6842323079045386
F1 score:  0.612636197875275
Confusion matrix for label toxic:
[[54806  3082]
 [ 1502  4588]]
Confusion matrix for label severe_toxic:
[[63268   343]
 [  209   158]]
Confusion matrix for label obscene:
[[58263  2024]
 [  965  2726]]
Confusion matrix for label threat:
[[63426   341]
 [  150    61]]
Confusion matrix for label insult:
[[58812  1739]
 [ 1222  2205]]
Confusion matrix for label identity_hate:
[[62931   335]
 [  530   182]]
Logarithmic Loss:  0.33110831807467267
ROC AUC score:  0.9556995692510866


### Baseline Gradient Boosting using LabelPowerset

In [70]:
classifier_powerset_gb = LabelPowerset(LGBMClassifier(random_state=0))
# train
classifier_powerset_gb.fit(selected_train[features], selected_train[labels])
# predict
predictions_powerset_gb = classifier_powerset_gb.predict(selected_train[features])

In [71]:
# predict probability
predictions_proba_powerset_gb = classifier_powerset_gb.predict_proba(selected_train[features])

In [72]:
# model evaluation
get_evaluation_score(selected_train[labels], predictions_powerset_gb, predictions_proba_powerset_gb)

Accuracy score:  0.7406358298187014
Precision score:  0.09740660064706258
Recall score:  0.17445438486523449
F1 score:  0.12454536249158364
Confusion matrix for label toxic:
[[122517  21760]
 [ 12063   3231]]
Confusion matrix for label severe_toxic:
[[153105   4871]
 [  1563     32]]
Confusion matrix for label obscene:
[[136712  14410]
 [  6960   1489]]
Confusion matrix for label threat:
[[151307   7786]
 [   474      4]]
Confusion matrix for label insult:
[[136507  15187]
 [  6551   1326]]
Confusion matrix for label identity_hate:
[[149986   8180]
 [  1364     41]]
Logarithmic Loss:  0.8053150175420156
ROC AUC score:  0.5289704923713838


In [73]:
# evaluation on test data
predictions_powerset_gb_test = classifier_powerset_gb.predict(selected_test[features])
predictions_proba_powerset_gb_test = classifier_powerset_gb.predict_proba(selected_test[features])
get_evaluation_score(selected_test[labels], predictions_powerset_gb_test, predictions_proba_powerset_gb_test)

Accuracy score:  0.7441307949607677
Precision score:  0.10160540877173552
Recall score:  0.1799558559801352
F1 score:  0.12966069946135153
Confusion matrix for label toxic:
[[48915  8973]
 [ 4753  1337]]
Confusion matrix for label severe_toxic:
[[61842  1769]
 [  362     5]]
Confusion matrix for label obscene:
[[54521  5766]
 [ 3037   654]]
Confusion matrix for label threat:
[[60954  2813]
 [  210     1]]
Confusion matrix for label insult:
[[54379  6172]
 [ 2828   599]]
Confusion matrix for label identity_hate:
[[60239  3027]
 [  699    13]]
Logarithmic Loss:  0.8015398931821524
ROC AUC score:  0.5309322488614058


### Baseline Gradient Boosting using OneVsRest

In [116]:
# Using pipeline for applying lgbm and one vs rest classifier
LGBM_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LGBMClassifier(), n_jobs=-1)),
            ])
for category in labels:
    print('**Processing {} comments...**'.format(category))
    
    # Training logistic regression model on train data
    LGBM_pipeline.fit(selected_train[features], selected_train[category])
    
    # calculating test accuracy (on test dataset)
    prediction = LGBM_pipeline.predict(selected_test[features])
    print('Test accuracy is {}'.format(accuracy_score(selected_test[category], prediction)))
    print('Test precision is {}'.format(precision_score(selected_test[category], prediction)))
    print('Test f1-score is {}'.format(f1_score(selected_test[category], prediction)))
    print('Test recall is {}'.format(recall_score(selected_test[category], prediction)))
    print(confusion_matrix(selected_test[category], prediction))
    print('Test logarithmic loss is {}'.format(log_loss(selected_test[category], prediction)))
    print('Test ROC-AUC score is {}'.format(roc_auc_score(selected_test[category], prediction)))
    print("\n")

**Processing toxic comments...**
Test accuracy is 0.9283503704398387
Test precision is 0.598174706649283
Test f1-score is 0.666860465116279
Test recall is 0.7533661740558292
[[54806  3082]
 [ 1502  4588]]
Test logarithmic loss is 2.5825144133250966
Test ROC-AUC score is 0.8500627166575443


**Processing severe_toxic comments...**
Test accuracy is 0.9909812748132171
Test precision is 0.2916666666666667
Test f1-score is 0.3375430539609644
Test recall is 0.40054495912806537
[[63254   357]
 [  220   147]]
Test logarithmic loss is 0.32506780464410595
Test ROC-AUC score is 0.6974663611253978


**Processing obscene comments...**
Test accuracy is 0.9568914314295539
Test precision is 0.6083120501509172
Test f1-score is 0.655163790947737
Test recall is 0.7098347331346518
[[58600  1687]
 [ 1071  2620]]
Test logarithmic loss is 1.5537903036541485
Test ROC-AUC score is 0.840925958801141


**Processing threat comments...**
Test accuracy is 0.9915283378661415
Test precision is 0.12471655328798185
Tes

## Naive Bayes (GaussianNB)

### Baseline Naive Bayes using BinaryRelevance

In [97]:
import warnings
warnings.filterwarnings("ignore")

classifier_nb = BinaryRelevance(GaussianNB())
# train
classifier_nb.fit(selected_train[features], selected_train[labels])
# predict
predictions_nb = classifier_nb.predict(selected_train[features])

In [98]:
# predict probability
predictions_proba_nb = classifier_nb.predict_proba(selected_train[features])

In [99]:
# model evaluation
get_evaluation_score(selected_train[labels], predictions_nb, predictions_proba_nb)

Accuracy score:  0.8792637759993984
Precision score:  0.5580383909328479
Recall score:  0.6486124565502308
F1 score:  0.577292473981047
Confusion matrix for label toxic:
[[140351   3926]
 [  5871   9423]]
Confusion matrix for label severe_toxic:
[[152083   5893]
 [   292   1303]]
Confusion matrix for label obscene:
[[146146   4976]
 [  2527   5922]]
Confusion matrix for label threat:
[[151647   7446]
 [   181    297]]
Confusion matrix for label insult:
[[146219   5475]
 [  2896   4981]]
Confusion matrix for label identity_hate:
[[150542   7624]
 [   566    839]]
Logarithmic Loss:  0.7660158339188156
ROC AUC score:  0.9312601005621751


In [100]:
# evaluation on test data
predictions_nb_test = classifier_nb.predict(selected_test[features])
predictions_proba_nb_test = classifier_nb.predict_proba(selected_test[features])
get_evaluation_score(selected_test[labels], predictions_nb_test, predictions_proba_nb_test)

Accuracy score:  0.8374284910437964
Precision score:  0.39914770987296333
Recall score:  0.7291350531107739
F1 score:  0.5031102162203966
Confusion matrix for label toxic:
[[53390  4498]
 [ 1571  4519]]
Confusion matrix for label severe_toxic:
[[59159  4452]
 [   68   299]]
Confusion matrix for label obscene:
[[55923  4364]
 [  942  2749]]
Confusion matrix for label threat:
[[59377  4390]
 [   53   158]]
Confusion matrix for label insult:
[[56254  4297]
 [ 1066  2361]]
Confusion matrix for label identity_hate:
[[58234  5032]
 [  227   485]]
Logarithmic Loss:  0.70881977326765
ROC AUC score:  0.9267265814152312


### Baseline  Naive Bayes using ClassifierChain

In [106]:
import warnings
warnings.filterwarnings("ignore")

classifier_chain_nb = ClassifierChain(GaussianNB())
# train
classifier_chain_nb.fit(selected_train[features], selected_train[labels])
# predict
predictions_chain_nb = classifier_chain_nb.predict(selected_train[features])

In [107]:
# predict probability
predictions_proba_chain_nb = classifier_chain_nb.predict_proba(selected_train[features])

In [108]:
# model evaluation
get_evaluation_score(selected_train[labels], predictions_chain_nb, predictions_proba_chain_nb)

Accuracy score:  0.8788125661931052
Precision score:  0.5565655674617817
Recall score:  0.67795885805459
F1 score:  0.5823247163434749
Confusion matrix for label toxic:
[[140351   3926]
 [  5871   9423]]
Confusion matrix for label severe_toxic:
[[149947   8029]
 [   182   1413]]
Confusion matrix for label obscene:
[[145923   5199]
 [  2302   6147]]
Confusion matrix for label threat:
[[149029  10064]
 [   143    335]]
Confusion matrix for label insult:
[[145785   5909]
 [  2440   5437]]
Confusion matrix for label identity_hate:
[[148149  10017]
 [   365   1040]]
Logarithmic Loss:  0.6688595094005957
ROC AUC score:  0.931072059247107


In [109]:
# evaluation on test data
predictions_chain_nb_test = classifier_chain_nb.predict(selected_test[features])
predictions_proba_chain_nb_test = classifier_chain_nb.predict_proba(selected_test[features])
get_evaluation_score(selected_test[labels], predictions_chain_nb_test, predictions_proba_chain_nb_test)

Accuracy score:  0.8373503391790929
Precision score:  0.394723171539833
Recall score:  0.7618981928541868
F1 score:  0.5036299318614065
Confusion matrix for label toxic:
[[53390  4498]
 [ 1571  4519]]
Confusion matrix for label severe_toxic:
[[57894  5717]
 [   44   323]]
Confusion matrix for label obscene:
[[55651  4636]
 [  842  2849]]
Confusion matrix for label threat:
[[57428  6339]
 [   34   177]]
Confusion matrix for label insult:
[[55645  4906]
 [  848  2579]]
Confusion matrix for label identity_hate:
[[56704  6562]
 [  113   599]]
Logarithmic Loss:  0.6631063220237972
ROC AUC score:  0.9251122828319055


### Baseline  Naive Bayes using LabelPowerSet

In [111]:
import warnings
warnings.filterwarnings("ignore")

classifier_power_nb = LabelPowerset(GaussianNB())
# train
classifier_power_nb.fit(selected_train[features], selected_train[labels])
# predict
predictions_power_nb = classifier_power_nb.predict(selected_train[features])

In [112]:
# predict probability
predictions_proba_power_nb = classifier_power_nb.predict_proba(selected_train[features])

In [113]:
# model evaluation
get_evaluation_score(selected_train[labels], predictions_power_nb, predictions_proba_power_nb)

Accuracy score:  0.8656021457533011
Precision score:  0.5987943558266151
Recall score:  0.5372670807453416
F1 score:  0.553592035258963
Confusion matrix for label toxic:
[[140607   3670]
 [  8174   7120]]
Confusion matrix for label severe_toxic:
[[156415   1561]
 [   857    738]]
Confusion matrix for label obscene:
[[147546   3576]
 [  2595   5854]]
Confusion matrix for label threat:
[[154431   4662]
 [   189    289]]
Confusion matrix for label insult:
[[149165   2529]
 [  3482   4395]]
Confusion matrix for label identity_hate:
[[153987   4179]
 [   944    461]]
Logarithmic Loss:  0.7372777504710718
ROC AUC score:  0.9422719716163704


In [114]:
# evaluation on test data
predictions_power_nb_test = classifier_power_nb.predict(selected_test[features])
predictions_proba_power_nb_test = classifier_power_nb.predict_proba(selected_test[features])
get_evaluation_score(selected_test[labels], predictions_power_nb_test, predictions_proba_power_nb_test)

Accuracy score:  0.8111069430116603
Precision score:  0.43116617623536224
Recall score:  0.5779417850738033
F1 score:  0.4836571435450641
Confusion matrix for label toxic:
[[54235  3653]
 [ 2620  3470]]
Confusion matrix for label severe_toxic:
[[62171  1440]
 [  200   167]]
Confusion matrix for label obscene:
[[56730  3557]
 [ 1189  2502]]
Confusion matrix for label threat:
[[61769  1998]
 [   87   124]]
Confusion matrix for label insult:
[[58579  1972]
 [ 1594  1833]]
Confusion matrix for label identity_hate:
[[60119  3147]
 [  429   283]]
Logarithmic Loss:  0.6426103750949465
ROC AUC score:  0.9285302105506413


### Baseline  Naive Bayes using OneVsRest

In [117]:
# Using pipeline for applying naive bayes and one vs rest classifier
NaiveBayes_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(GaussianNB(), n_jobs=-1)),
            ])
for category in labels:
    print('**Processing {} comments...**'.format(category))
    
    # Training logistic regression model on train data
    NaiveBayes_pipeline.fit(selected_train[features], selected_train[category])
    
    # calculating test accuracy (on test dataset)
    prediction = NaiveBayes_pipeline.predict(selected_test[features])
    print('Test accuracy is {}'.format(accuracy_score(selected_test[category], prediction)))
    print('Test precision is {}'.format(precision_score(selected_test[category], prediction)))
    print('Test f1-score is {}'.format(f1_score(selected_test[category], prediction)))
    print('Test recall is {}'.format(recall_score(selected_test[category], prediction)))
    print(confusion_matrix(selected_test[category], prediction))
    print('Test logarithmic loss is {}'.format(log_loss(selected_test[category], prediction)))
    print('Test ROC-AUC score is {}'.format(roc_auc_score(selected_test[category], prediction)))
    print("\n")

**Processing toxic comments...**
Test accuracy is 0.9051392666229017
Test precision is 0.5011644671176666
Test f1-score is 0.598265704640233
Test recall is 0.7420361247947455
[[53390  4498]
 [ 1571  4519]]
Test logarithmic loss is 3.4191273940815905
Test ROC-AUC score is 0.8321671779308166


**Processing severe_toxic comments...**
Test accuracy is 0.9293507143080434
Test precision is 0.06293411913281415
Test f1-score is 0.11684251660805002
Test recall is 0.8147138964577657
[[59159  4452]
 [   68   299]]
Test logarithmic loss is 2.546458365669597
Test ROC-AUC score is 0.8723630006412014


**Processing obscene comments...**
Test accuracy is 0.9170652411766544
Test precision is 0.38647546745395756
Test f1-score is 0.5088855979266939
Test recall is 0.7447846112164725
[[55923  4364]
 [  942  2749]]
Test logarithmic loss is 2.989271700938692
Test ROC-AUC score is 0.836198764712189


**Processing threat comments...**
Test accuracy is 0.9305542530244771
Test precision is 0.034740545294635
Test

### -- end -- 